# Credit Risk Resampling Techniques

In [39]:
import warnings
warnings.filterwarnings('ignore')

In [40]:
import numpy as np
import pandas as pd
from pathlib import Path
from collections import Counter

# Read the CSV into DataFrame

In [41]:
# Load the data
file_path = Path('Resources/lending_data.csv')
df = pd.read_csv(file_path)
df.head()

,loan_size,interest_rate,homeowner,borrower_income,debt_to_income,num_of_accounts,derogatory_marks,total_debt,loan_status
0,10700.0,7.672,own,52800,0.431818,5,1,22800,low_risk
1,8400.0,6.692,own,43600,0.311927,3,0,13600,low_risk
2,9000.0,6.963,rent,46100,0.349241,3,0,16100,low_risk
3,10700.0,7.664,own,52700,0.430740,5,1,22700,low_risk
4,10800.0,7.698,mortgage,53000,0.433962,5,1,23000,low_risk


In [42]:
dummies = pd.get_dummies(df['homeowner'])
df = pd.concat([df, dummies], axis = 1)
df = df.drop(columns = ('homeowner'), axis = 1)
df.head()

,loan_size,interest_rate,borrower_income,debt_to_income,num_of_accounts,derogatory_marks,total_debt,loan_status,mortgage,own,rent
0,10700.0,7.672,52800,0.431818,5,1,22800,low_risk,0,1,0
1,8400.0,6.692,43600,0.311927,3,0,13600,low_risk,0,1,0
2,9000.0,6.963,46100,0.349241,3,0,16100,low_risk,0,0,1
3,10700.0,7.664,52700,0.430740,5,1,22700,low_risk,0,1,0
4,10800.0,7.698,53000,0.433962,5,1,23000,low_risk,1,0,0


# Split the Data into Training and Testing

In [43]:
# Create our features
X = df.drop(columns = 'loan_status')

# Create our target
y = df['loan_status']

In [44]:
X.describe()

,loan_size,interest_rate,borrower_income,debt_to_income,num_of_accounts,derogatory_marks,total_debt,mortgage,own,rent
count,77536.000000,77536.000000,77536.000000,77536.000000,77536.000000,77536.000000,77536.000000,77536.000000,77536.000000,77536.000000
mean,9805.562577,7.292333,49221.949804,0.377318,3.826610,0.392308,19221.949804,0.497472,0.398911,0.103616
std,2093.223153,0.889495,8371.635077,0.081519,1.904426,0.582086,8371.635077,0.499997,0.489678,0.304764
min,5000.000000,5.250000,30000.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,8700.000000,6.825000,44800.000000,0.330357,3.000000,0.000000,14800.000000,0.000000,0.000000,0.000000
50%,9500.000000,7.172000,48100.000000,0.376299,4.000000,0.000000,18100.000000,0.000000,0.000000,0.000000
75%,10400.000000,7.528000,51400.000000,0.416342,4.000000,1.000000,21400.000000,1.000000,1.000000,0.000000
max,23800.000000,13.235000,105200.000000,0.714829,16.000000,3.000000,75200.000000,1.000000,1.000000,1.000000


In [45]:
# Check the balance of our target values
y.value_counts()

low_risk     75036
high_risk     2500
Name: loan_status, dtype: int64

In [46]:
# Create X_train, X_test, y_train, y_test
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test= train_test_split(X, 
                                                   y, 
                                                   random_state=1, 
                                                   stratify=y)
X_train.shape

(58152, 10)

## Data Pre-Processing

Scale the training and testing data using the `StandardScaler` from `sklearn`. Remember that when scaling the data, you only scale the features data (`X_train` and `X_testing`).

In [47]:
# Create the StandardScaler instance
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()

In [48]:
# Fit the Standard Scaler with the training data
# When fitting scaling functions, only train on the training dataset
X_scaler = scaler.fit(X_train)

In [49]:
# Scale the training and testing data
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

# Simple Logistic Regression

In [50]:
from sklearn.linear_model import LogisticRegression
model = LogisticRegression(solver='lbfgs', random_state=1)
model.fit(X_train, y_train)

LogisticRegression(random_state=1)

In [51]:
# Calculated the balanced accuracy score
from sklearn.metrics import balanced_accuracy_score
y_pred = model.predict(X_test)
bas = balanced_accuracy_score(y_test, y_pred)
bas

0.9543211898288821

In [52]:
# Display the confusion matrix
from sklearn.metrics import confusion_matrix
confusion_matrix(y_test, y_pred)

array([[  571,    54],
       [   93, 18666]])

In [53]:
# Print the imbalanced classification report
from imblearn.metrics import classification_report_imbalanced
print(classification_report_imbalanced(y_test, y_pred))

                   pre       rec       spe        f1       geo       iba       sup

  high_risk       0.86      0.91      1.00      0.89      0.95      0.90       625
   low_risk       1.00      1.00      0.91      1.00      0.95      0.92     18759

avg / total       0.99      0.99      0.92      0.99      0.95      0.92     19384



# Oversampling

In this section, you will compare two oversampling algorithms to determine which algorithm results in the best performance. You will oversample the data using the naive random oversampling algorithm and the SMOTE algorithm. For each algorithm, be sure to complete the folliowing steps:

1. View the count of the target classes using `Counter` from the collections library. 
3. Use the resampled data to train a logistic regression model.
3. Calculate the balanced accuracy score from sklearn.metrics.
4. Print the confusion matrix from sklearn.metrics.
5. Generate a classication report using the `imbalanced_classification_report` from imbalanced-learn.

Note: Use a random state of 1 for each sampling algorithm to ensure consistency between tests

### Naive Random Oversampling

In [54]:
# Resample the training data with the RandomOversampler
from imblearn.over_sampling import RandomOverSampler

# View the count of target classes with Counter
ros = RandomOverSampler(random_state=1)
X_resampled_ros, y_resampled_ros = ros.fit_resample(X_train_scaled, y_train)
Counter(y_resampled_ros)

Counter({'low_risk': 56277, 'high_risk': 56277})

In [55]:
# Train the Logistic Regression model using the resampled data
model_ros = LogisticRegression(solver='lbfgs', random_state=1)
model_ros.fit(X_resampled_ros, y_resampled_ros)

LogisticRegression(random_state=1)

In [56]:
# Calculated the balanced accuracy score
y_pred_ros = model_ros.predict(X_test_scaled)

bas_ros=balanced_accuracy_score(y_test, y_pred_ros)
bas_ros

0.9946414201183431

In [57]:
# Display the confusion matrix
cm_ros = confusion_matrix(y_test, y_pred_ros)
cm_df_ros = pd.DataFrame(
    cm_ros, index=["Actual High Risk", "Actual Low Risk"], columns=["Predicted High Risk", "Predicted Low Risk"]
)
cm_df_ros

,Predicted High Risk,Predicted Low Risk
Actual High Risk,622,3
Actual Low Risk,111,18648


In [58]:
# Print the imbalanced classification report
print(classification_report_imbalanced(y_test, y_pred_ros))

                   pre       rec       spe        f1       geo       iba       sup

  high_risk       0.85      1.00      0.99      0.92      0.99      0.99       625
   low_risk       1.00      0.99      1.00      1.00      0.99      0.99     18759

avg / total       0.99      0.99      1.00      0.99      0.99      0.99     19384



### SMOTE Oversampling

In [59]:
# Resample the training data with SMOTE
from imblearn.over_sampling import SMOTE

X_resampled_smote, y_resampled_smote = SMOTE(random_state=1, sampling_strategy=1.0).fit_resample(
    X_train_scaled, y_train)
# View the count of target classes with Counter
Counter(y_resampled_smote)

Counter({'low_risk': 56277, 'high_risk': 56277})

In [60]:
# Train the Logistic Regression model using the resampled data
model_smote = LogisticRegression(solver='lbfgs', random_state=1)
model_smote.fit(X_resampled_smote, y_resampled_smote)

LogisticRegression(random_state=1)

In [61]:
# Calculated the balanced accuracy score
y_pred_smote = model_smote.predict(X_test_scaled)
bas_smote=balanced_accuracy_score(y_test, y_pred_smote)
bas_smote

0.9946680739911509

In [62]:
# Display the confusion matrix
cm_smote = confusion_matrix(y_test, y_pred_smote)
cm_df_smote = pd.DataFrame(
    cm_smote, index=["Actual High Risk", "Actual Low Risk"], columns=["Predicted High Risk", "Predicted Low Risk"]
)
cm_df_smote

,Predicted High Risk,Predicted Low Risk
Actual High Risk,622,3
Actual Low Risk,110,18649


In [63]:
# Print the imbalanced classification report
print(classification_report_imbalanced(y_test, y_pred_smote))

                   pre       rec       spe        f1       geo       iba       sup

  high_risk       0.85      1.00      0.99      0.92      0.99      0.99       625
   low_risk       1.00      0.99      1.00      1.00      0.99      0.99     18759

avg / total       0.99      0.99      1.00      0.99      0.99      0.99     19384



# Undersampling

In this section, you will test an undersampling algorithm to determine which algorithm results in the best performance compared to the oversampling algorithms above. You will undersample the data using the Cluster Centroids algorithm and complete the folliowing steps:

1. View the count of the target classes using `Counter` from the collections library. 
3. Use the resampled data to train a logistic regression model.
3. Calculate the balanced accuracy score from sklearn.metrics.
4. Display the confusion matrix from sklearn.metrics.
5. Generate a classication report using the `imbalanced_classification_report` from imbalanced-learn.

Note: Use a random state of 1 for each sampling algorithm to ensure consistency between tests

In [64]:
from imblearn.under_sampling import ClusterCentroids

In [65]:
# Resample the data using the ClusterCentroids resampler

cc = ClusterCentroids(random_state=1)
X_resampled_cc, y_resampled_cc = cc.fit_resample(X_train_scaled, y_train)
# View the count of target classes with Counter
from collections import Counter

Counter(y_resampled_cc)

Counter({'high_risk': 1875, 'low_risk': 1875})

In [66]:
# Train the Logistic Regression model using the resampled data
model_cc = LogisticRegression(solver='lbfgs', random_state=1)
model_cc.fit(X_resampled_cc, y_resampled_cc)

LogisticRegression(random_state=1)

In [67]:
# Calculate the balanced accuracy score
y_pred_cc = model_cc.predict(X_test_scaled)
bas_cc=balanced_accuracy_score(y_test, y_pred_cc)
print(bas_cc)

0.9932546511008049


In [68]:
# Display the confusion matrix
cm_cc = confusion_matrix(y_test, y_pred_cc)
cm_df_cc = pd.DataFrame(
    cm_cc, index=["Actual High Risk", "Actual Low Risk"], columns=["Predicted High Risk", "Predicted Low Risk"]
)
cm_df_cc

,Predicted High Risk,Predicted Low Risk
Actual High Risk,620,5
Actual Low Risk,103,18656


In [69]:
# Print the imbalanced classification report
print(classification_report_imbalanced(y_test, y_pred_cc))

                   pre       rec       spe        f1       geo       iba       sup

  high_risk       0.86      0.99      0.99      0.92      0.99      0.99       625
   low_risk       1.00      0.99      0.99      1.00      0.99      0.99     18759

avg / total       1.00      0.99      0.99      0.99      0.99      0.99     19384



# Combination (Over and Under) Sampling

In this section, you will test a combination over- and under-sampling algorithm to determine if the algorithm results in the best performance compared to the other sampling algorithms above. You will resample the data using the SMOTEENN algorithm and complete the folliowing steps:

1. View the count of the target classes using `Counter` from the collections library. 
3. Use the resampled data to train a logistic regression model.
3. Calculate the balanced accuracy score from sklearn.metrics.
4. Display the confusion matrix from sklearn.metrics.
5. Generate a classication report using the `imbalanced_classification_report` from imbalanced-learn.

Note: Use a random state of 1 for each sampling algorithm to ensure consistency between tests

In [70]:
# Resample the training data with SMOTEENN
from imblearn.combine import SMOTEENN

smote_enn = SMOTEENN(random_state=0)
X_resampled_se, y_resampled_se= smote_enn.fit_resample(X_train_scaled, y_train)

# View the count of target classes with Counter
Counter(y_resampled_se)

Counter({'high_risk': 55645, 'low_risk': 55932})

In [71]:
# Train the Logistic Regression model using the resampled data
from sklearn.linear_model import LogisticRegression
model_se = LogisticRegression(solver='lbfgs', random_state=1)
model_se.fit(X_resampled_se, y_resampled_se)

LogisticRegression(random_state=1)

In [72]:
# Calculate the balanced accuracy score
y_pred_se = model_se.predict(X_test_scaled)
bas_se=balanced_accuracy_score(y_test, y_pred_se)
print(bas_se)

0.9946680739911509


In [73]:
# Display the confusion matrix
cm_se = confusion_matrix(y_test, y_pred_se)
cm_df_se = pd.DataFrame(
    cm_se, index=["Actual High Risk", "Actual Low Risk"], columns=["Predicted High Risk", "Predicted Low Risk"]
)
cm_df_se

,Predicted High Risk,Predicted Low Risk
Actual High Risk,622,3
Actual Low Risk,110,18649


In [74]:
# Print the imbalanced classification report
from imblearn.metrics import classification_report_imbalanced
print(classification_report_imbalanced(y_test, y_pred_se))

                   pre       rec       spe        f1       geo       iba       sup

  high_risk       0.85      1.00      0.99      0.92      0.99      0.99       625
   low_risk       1.00      0.99      1.00      1.00      0.99      0.99     18759

avg / total       0.99      0.99      1.00      0.99      0.99      0.99     19384



# Final Questions

#### 1. Which model had the best balanced accuracy score?

In [76]:
print(f"Original data set's balanced accuracy score = {bas}")
print(f"-------------------------------------------------------------------")
print(f"Naive oversampling balanced accuracy score = {bas_ros} ")
print(f"SMOTE balanced accuracy score = {bas_smote} ")
print(f"SMOTEEN balanced accuracy score = {bas_se} ")

Original data set's balanced accuracy score = 0.9543211898288821
-------------------------------------------------------------------
Naive oversampling balanced accuracy score = 0.9946414201183431 
SMOTE balanced accuracy score = 0.9946680739911509 
SMOTEEN balanced accuracy score = 0.9946680739911509 


The Naive Oversampling, SMOTE, and SMOTEENN all had the same balanced score. Each of which was higher than the original data set's balanced score accuracy.  

#### 2. Which model had the best recall score?

The Naive, SMOTE, and SMOTEEN all had the same recall score. High-Risk= 1.00, Low-Risk= 0.99, and Average Total= 0.99. The Simple Logistic Regression had a recall score of - High-Risk= 0.91, Low-Risk= 1.00, Average Total=0.99 

#### 3. Which model had the best geometric mean score?

All models had the same geometric mean score. High-Risk= 0.99, Low-Risk= 0.99, Average Total= 0.99. The Simple Logistic Regression model had a geometric mean of- High-Risk= 0.95, Low-Risk= 0.95, Average Total= 0.95